In [41]:
from contextlib import contextmanager
from pathlib import Path
from typing import Optional, Union

import tables

In [55]:
import csv

with open('check.csv', 'x') as f:
    pass

In [21]:
import re
import inspect
import logging
import typing
from dataclasses import dataclass
from datetime import datetime
from typing import Literal
from threading import Lock

from pathlib import Path

from rich.logging import RichHandler

from logging.handlers import RotatingFileHandler

LOGLEVELS = Literal["DEBUG", "INFO", "WARNING", "ERROR"]

_INIT_LOCK = Lock() # type: Lock
_LOGGERS = [] # type: list

def init_logger(
    instance=None, module_name=None, class_name=None, object_name=None
) -> logging.Logger:


# --------------------------------------------------
    # gather variables
    # --------------------------------------------------

    if instance is not None:
        # get name of module_name without prefixed autopilot
        # eg passed autopilot.hardware.gpio.Digital_In -> hardware.gpio
        # filtering leading 'autopilot' from string

        module_name = instance.__module__
        if "__main__" in module_name:
            # awkward workaround to get module name of __main__ run objects
            mod_obj = inspect.getmodule(instance)
            try:
                mod_suffix  = inspect.getmodulename(inspect.getmodule(instance).__file__)
                module_name = '.'.join([mod_obj.__package__, mod_suffix])
            except AttributeError:
                # when running interactively or from a plugin, __main__ does not have __file__
                module_name = "__main__"

        module_name = re.sub('^NeuRPi.', '', module_name)

        class_name = instance.__class__.__name__

        if hasattr(instance, 'id'):
            object_name = str(instance.id)
        elif hasattr(instance, 'name'):
            object_name = str(instance.name)
        else:
            object_name = None



        # --------------------------------------------------
        # check if logger needs to be made, or exists already
        # --------------------------------------------------
    elif not any((module_name, class_name, object_name)):
        raise ValueError('Need to either give an object to create a logger for, or one of module_name, class_name, or object_name')


    # get name of logger to get
    logger_name_pieces = [v for v in (module_name, class_name, object_name) if v is not None]
    logger_name = '.'.join(logger_name_pieces)

    # trim __ from logger names, linux don't like to make things like that
    # re.sub(r"^\_\_")

    # --------------------------------------------------
    # if new logger must be made, make it, otherwise just return existing logger
    # --------------------------------------------------

    # use a lock to prevent loggers from being double-created, just to be extra careful
    with globals()['_INIT_LOCK']:

        # check if something starting with module_name already exists in loggers
        MAKE_NEW = False
        if not any([test_logger == module_name for test_logger in globals()['_LOGGERS']]):
            MAKE_NEW = True

        if MAKE_NEW:
            parent_logger = logging.getLogger(module_name)
            loglevel = 'WARNING' #getattr(logging, prefs.get('LOGLEVEL'))
            parent_logger.setLevel(loglevel)

            # make formatter that includes name
            log_formatter = logging.Formatter("[%(asctime)s] %(levelname)s [%(name)s]: %(message)s")

            ## file handler
            # base filename is the module_name + '.log
            base_filename = Path(module_name + '.log')

            fh = _file_handler(base_filename)
            fh.setLevel(loglevel)
            fh.setFormatter(log_formatter)
            parent_logger.addHandler(fh)

            # rich logging handler for stdout
            parent_logger.addHandler(_rich_handler())

            # if our parent is the rootlogger, disable propagation to avoid printing to stdout
            if isinstance(parent_logger.parent, logging.RootLogger):
                parent_logger.propagate = False

            ## log creation
            globals()['_LOGGERS'].append(module_name)
            parent_logger.debug(f'parent, module-level logger created: {module_name}')

        logger = logging.getLogger(logger_name)
        if logger_name not in globals()['_LOGGERS']:
        # logger.addHandler(_rich_handler())
            logger.debug(f"Logger created: {logger_name}")
            globals()['_LOGGERS'].append(logger_name)

    return logger


def _rich_handler() -> RichHandler:
    rich_handler = RichHandler(rich_tracebacks=True, markup=True)
    rich_formatter = logging.Formatter(
        "[bold green]\[%(name)s][/bold green] %(message)s",
        datefmt='[%y-%m-%dT%H:%M:%S]'
    )
    rich_handler.setFormatter(rich_formatter)
    return rich_handler

def _file_handler(base_filename: Path) -> RotatingFileHandler:
    # if directory doesn't exist, try to make it
    if not base_filename.parent.exists():
        base_filename.parent.mkdir(parents=True, exist_ok=True)

    fh = RotatingFileHandler(
        str(base_filename),
        mode='a',
        maxBytes=int(20*(2**20) ),
        backupCount=int(4)
        )
    return fh

from NeuRPi.hardware.arduino import Arduino
a = Arduino()
b = init_logger(a)

In [52]:
class SubjectStructure():
    """
    Structure of the :class:`.Subject` class's hdf5 file
    """

    info = H5F_Group(path="/info", title="Subject Biographical Information")
    data = H5F_Group(
        path="/data", filters=tables.Filters(complevel=6, complib="blosc:lz4")
    )
    protocol = H5F_Group(
        path="/protocol", title="Metadata for the currently assigned protocol"
    )
    history = H5F_Group(
        path="/data/task_name/phase/",
        children=[
            H5F_Group(path="/history/past_protocols", title="Past Protocol Files"),
            _Hash_Table(path="/history/hashes", title="Git commit hash history"),
            _History_Table(path="/history/history", title="Change History"),
            _Weight_Table(path="/history/weights", title="Subject Weights"),
        ],
    )

AttributeError: 'H5F_Group' object has no attribute '_init_logger'

In [51]:
class H5F_Group(H5F_Node):
    """
    Description of a pytables group and its location
    """
    

    children: Optional[List[Union[H5F_Node, 'H5F_Group']]] = None
        
    def make(self, h5f:tables.file.File):
        """
        Make the group, if it doesn't already exist.

        If it exists, do nothing

        Args:
            h5f (:class:`tables.file.File`): The file to create the table in
        """

        try:
            node = h5f.get_node(self.path)
            # if no exception, already exists
            if not isinstance(node, tables.group.Group):
                raise ValueError(f'{self.path} already exists, but it isnt a group! instead its a {type(node)}')
        except tables.exceptions.NoSuchNodeError:
            group = h5f.create_group(self.parent, self.name,
                             title=self.title, createparents=True,
                             filters=self.filters)
            self._logger.debug(f"Made group {'/'.join([self.parent, self.name])}")
            if self.attrs is not None:
                group._v_attrs.update(self.attrs)

        if self.children is not None:
            for c in self.children:
                c.make(h5f)
        h5f.flush()

In [50]:

from abc import abstractmethod

class H5F_Node(Node):
    """
    Base class for H5F Nodes
    """
    path:str
    title:Optional[str]=''
    filters:Optional[tables.filters.Filters]=None
    attrs:Optional[dict]=None

    def __init__(self, **data):
        self._init_logger()
        super().__init__(**data)

    @property
    def parent(self) -> str:
        """
        The parent node under which this node hangs.

        Eg. if ``self.path`` is ``/this/is/my/path``, then
        parent will be ``/this/is/my``

        Returns:
            str
        """
        return '/'.join(self.path.split('/')[:-1])

    @property
    def name(self) -> str:
        """
        Our path without :attr:`.parent`

        Returns:
            str
        """
        return self.path.split('/')[-1]

    @abstractmethod
    def make(self, h5f:tables.file.File):
        """
        Abstract method to make whatever this node is
        """

    class Config:
        arbitrary_types_allowed = True

In [42]:
"""
Base classes for data models -- the ``Data`` class itself.
"""
import typing
from datetime import datetime
from typing import List, Optional

import pandas as pd
import tables


class Data():
    """
    The top-level container for Data.

    Subtypes will define more specific formats and uses of data, but this is the most general
    form used to represent the type and meaning of data.

    The Data class is not intended to contain individual fields, but collections of data that are collected
    as a unit, whether that be a video frame along with its timestamp and encoding, or a single trial of behavioral data.

    This class is also generally not intended to be used for the literal transport of data when performance is
    necessary: this class by default does type validation on instantiation that takes time (see the `construct <https://pydantic-docs.helpmanual.io/usage/models/#creating-models-without-validation>`_
    method for validation-less creation). It is usually more to specify the type, grouping, and annotation for
    a given unit of data -- though users should feel free to dump their data in a :class:`.Data` object if
    it is not particularly performance sensitive.
    """


class Table(Data):
    """
    Tabular data: each field will have multiple values -- in particular an equal number across fields.

    Used for trialwise data, and can be used to create pytables descriptions.

    .. todo::

        To make this usable as a live container of data, the fields need to be declared as Lists (eg. instead of just
        declaring something an ``int``, it must be specified as a ``List[int]`` to pass validation. We should expand this
        model to relax that constraint and effectively treat every field as containing a list of values.
    """

    @classmethod
    def to_pytables_description(cls) -> typing.Type[tables.IsDescription]:
        """
        Convert the fields of this table to a pytables description.

        See :func:`~.interfaces.tables.model_to_description`
        """
        from autopilot.data.interfaces.tables import model_to_description

        return model_to_description(cls)

    @classmethod
    def from_pytables_description(
        cls, description: typing.Type[tables.IsDescription]
    ) -> "Table":
        """
        Create an instance of a table from a pytables description

        See :func:`~.interfaces.tables.description_to_model`

        Args:
            description (:class:`tables.IsDescription`): A Pytables description
        """
        from autopilot.data.interfaces.tables import description_to_model

        return description_to_model(description, cls)

    def to_df(self) -> pd.DataFrame:
        """
        Create a dataframe from the lists of fields

        Returns:
            :class:`pandas.DataFrame`
        """
        return pd.DataFrame(self.dict())


class Attributes(Data):
    """
    A set of attributes that is intended to have a single representation per usage:
    eg. a subject has a single set of biographical information.

    Useful to specify a particular type of storage that doesn't need to include variable
    numbers of each field (eg. the tables interface stores attribute objects as metadata on a node, rather than as a table).
    """


class Schema():
    """
    A special type of type intended to be a representation of an
    abstract structure/schema of data, rather than a live container of
    data objects themselves. This class is used for constructing data containers,
    translating between formats, etc. rather than momentary data handling
    """


class Node():
    """
    Abstract representation of a Node in a treelike or linked data structure.
    This should be extended by interfaces when relevant and needed to implement
    an abstract representation of their structure.

    This class purposely lacks structure like a path or parents pending further
    usage in interfaces to see what would be the best means of implementing them.
    """


class Group():
    """
    A generic representation of a "Group" if present in a given interface.
    Useful for when, for example in a given container format you want to
    make an empty group that will be filled later, or one that has to be
    present for syntactic correctness.

    A children attribute is present because it is definitive of groups, but
    should be overridden by interfaces that use it.
    """

    children: Optional[List[Node]] = None


BASE_TYPES = (bool, int, float, str, bytes, datetime)
"""
Base Python types that should be suppported by every interface
"""


'\nBase Python types that should be suppported by every interface\n'